In [1]:
import numpy as np
batch = 20
dir = "CEAexec\\CEAexec-win\\"
infile = "template.inp"

# **********Step 1: Create CEA input.inp files***********

rang = np.array([[1, 1.25, 1.5, 2, 2.5, 3, 4, 5, 6.25, 8]])
P = rang * np.array([[10**x for x in range(-3,3)]]).transpose()
P = P.flatten()
T1 = np.arange(100, 1e3, 10, dtype=int)
T2 = np.arange(1e3, 1e4, 50, dtype=int)
T = np.concatenate((T1, T2))
fnames = []

for i in range(int(np.ceil(len(P)/batch))):
    for j in range(int(np.ceil(len(T)/batch))):
        Pvals = P[i*batch:min(len(P),(i+1)*batch)]
        Tvals = T[j*batch:min(len(T),(j+1)*batch)]
        Pstr = ','.join([str(pi) for pi in Pvals.tolist()])
        Tstr = ','.join([str(ti) for ti in Tvals.tolist()])
        with open(dir+infile, mode='r') as fread:
            contents = fread.read()
        contents = contents.replace("<PYPRES>", Pstr)
        contents = contents.replace("<PYTEMP>", Tstr)
        writename = f"AUTOFILE_P{i}T{j}"
        fnames.append(writename)
        with open(dir+writename+'.inp', mode='w') as fwrite:
            fwrite.write(contents)

In [2]:
# **********Step 2: Run CEA to produce output.out files***********
import os
import subprocess as sp
if not ("CEAexec-win" in os.getcwd()):
    os.chdir("C:/Users/david/OneDrive - Drexel University/Documents/GitHub/CFE/CEAexec/CEAexec-win")
for file in fnames:
    sub = sp.run(["echo", f"{file}", "|", "FCEA2.exe"], stdout=sp.PIPE, stderr=sp.PIPE, shell=True)


In [3]:
# **********Step 3: Read output files into dictionary ***********

thermo = {"P, BAR":[], "T, K":[], "RHO, KG/CU M":[], "H, KJ/KG":[], "U, KJ/KG":[], 
          "G, KJ/KG":[], "S, KJ/(KG)(K)":[], "Cp, KJ/(KG)(K)":[], "M, (1/n)":[], "AMMAs":[]}
transpheader = "WITH FROZEN REACTIONS"
transp = {"Cp, KJ/(KG)(K)":[], "CONDUCTIVITY":[],"PRANDTL NUMBER":[]}

# iterate over output files
for f in fnames:

    # open each file
    with open(f+".OUT") as file:
        lines = file.readlines()

    # iterate over each line of the file
    for textline in lines:

        # go through thermodynamic properties
        for dictkey, dictvalue in thermo.items():
            # Check if the property is references in that line
            if dictkey in textline:
                if len(textline)>16:
                    numtext = textline[16:]
                    columns=[]
                    for i in range(0,len(numtext)-8,9):
                        columns.append(numtext[i:i+9])
                else:
                    continue
                # start 
                for property_text in columns:
                    if property_text[-2]=='-' or property_text[-2]==' ':
                        propval = float(property_text[:-2])*10**float(property_text[-2:])
                    else:
                        propval = float(property_text)
                    dictvalue.append(propval)
                


In [4]:
# **********Step 4: Organize dictionary into arrays and write to CSVs ***********
import csv

# set up first row and first column containing pressure and temperature values
col1 = np.zeros((len(T),1))
col1[:,0]=T
row1 = np.zeros((1,len(P)+1))
row1[0,1:]=P

# loop through each property
for key in thermo.keys():
    # create an empty array for property
    tab = np.zeros((len(T), len(P)))
    tab[:] = np.nan

    # do not generate pressure and temperature tables as these serve as our axes
    if key!="P, BAR" and key!="T, K":

        # loop through every value of the property
        for i in range(len(thermo["P, BAR"])):
            P_idx = np.where(P==thermo["P, BAR"][i])
            T_idx = np.where(T==thermo["T, K"][i])
            tab[T_idx, P_idx] = thermo[key][i]
        print(tab.shape)
        tab = np.concatenate((col1,tab), axis=1)
        tab = np.concatenate((row1,tab))
        with open(f"{key[0]}.csv", mode='w', newline='') as csvfile:
            writer = csv.writer(csvfile)

            writer.writerows(tab)

    

#Ptable = [[P[i*nT+j] for j in range(nP)] for i in range(nT)]
table = lambda x: np.array([[x[i*nT+j] for j in range(nP)] for i in range(nT)])


(270, 60)
(270, 60)
(270, 60)
(270, 60)
(270, 60)
(270, 60)
(270, 60)
